### Задание 1.1

Пусть прибыль в вашей компании выражается функцией, которая зависит от параметра x — количества производимых товаров

Найдите максимально возможную прибыль, которую вы можете получить, варьируя количество произведённых товаров.

In [36]:
from sympy import *

In [37]:
x = Symbol('x')
f = -x**4 + 6 * x**3 - 4 * x**2 + 80
f 

-x**4 + 6*x**3 - 4*x**2 + 80

Нахожу первую производную

In [38]:
f_diff = f.diff(x)
f_diff

-4*x**3 + 18*x**2 - 8*x

Нахожу значения х при которых производная равна нулю

In [39]:
eq = Eq(f_diff, 0)
eq

Eq(-4*x**3 + 18*x**2 - 8*x, 0)

In [40]:
sol = solve(eq, x)
sol

[0, 1/2, 4]

Располагаю найденные значения x на координатной оси.

Опеределяю значения(знак) производной на каждом из участков.

Исходя из этого определяю какие экстремумы являются максимумами, а какие минимумами.

Значение производной на отрезках интервала

In [41]:
print('Значение производной до 0:', f_diff.subs(x, -1))
print('Значение производной от 0 до 1/2:', f_diff.subs(x, 0.1))
print('Значение производной от 1/2 до 4:', f_diff.subs(x, 2))
print('Значение производной от 4 до бесконечности:', f_diff.subs(x, 5))

Значение производной до 0: 30
Значение производной от 0 до 1/2: -0.624000000000000
Значение производной от 1/2 до 4: 24
Значение производной от 4 до бесконечности: -90


По смене знака производной с + на - обнаружил два максимума с коордиданами x = 0, 4

Считаю значение функции для этих максимумов и определяю максимум изначальной функции f

In [42]:
f.subs(x, 0)

80

In [43]:
f.subs(x, 4)

144

### Задание 1.4

Вам необходимо оптимизировать расход бумаги в принтере, который зависит от часов работы двух ключевых сотрудников исходя из функции f.

Где `x` — количество часов работы 1-ого сотрудника, `y` — 2-ого сотрудника.

Однако вы должны оплатить этим сотрудникам ровно 20 часов работы в сумме.

Какое наименьшее количество листов бумаги вы сможете потратить в таком случае?

Помните, что количество часов работы должно быть целым, поэтому, прежде чем вычислять итоговый результат, округлите часы работы до целого.

In [1]:
from sympy import *

In [2]:
x, y, l = symbols('x y lambda')
f = x**2 + 2 * y**2
f

x**2 + 2*y**2

Функция ограничений

In [46]:
g = x + y - 20
g

x + y - 20

Использую метод Лагранжа

In [47]:
L = f + l * g
L

lambda*(x + y - 20) + x**2 + 2*y**2

Определение частных производных по x, y, l

In [48]:
L_diff_x = L.diff(x)
L_diff_x

lambda + 2*x

In [49]:
L_diff_y = L.diff(y)
L_diff_y

lambda + 4*y

In [50]:
L_diff_l = L.diff(l)
L_diff_l

x + y - 20

Составляю систем уравнений приравнивая частные производные к нулю

In [51]:
eq0 = Eq(L_diff_x, 0)
eq1 = Eq(L_diff_y, 0)
eq2 = Eq(L_diff_l, 0)

sol = solve([eq0, eq1, eq2], [x, y, l])

Получаю значения x и y для заданного ограничения

In [52]:
sol

{lambda: -80/3, x: 40/3, y: 20/3}

Округляю часы работы до целых

In [53]:
round(40/3)

13

In [54]:
round(20/3)

7

Вычисляю основную зависимость исходя из найденных часов работы

In [55]:
f.subs({x: 13, y: 7})

267

### Задание 2.7

Давайте потренируемся применять стохастический градиентный спуск для решения задачи линейной регрессии. 

Мы уже рассмотрели его реализацию «с нуля», однако для решения практических задач можно использовать готовые библиотеки.

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns

# Загрузите стандартный датасет об алмазах из библиотеки Seaborn
df = sns.load_dataset('diamonds')

In [57]:
# Удалите часть признаков
df.drop(['depth', 'table', 'x', 'y', 'z'], axis=1, inplace=True)

In [58]:
# Закодируйте категориальные признаки
df = pd.get_dummies(df, drop_first=True)

In [59]:
# Логарифмируйте признаки
df['carat'] = np.log(1 + df['carat'])
df['price'] = np.log(1 + df['price'])

In [60]:
# Определите целевую переменную и предикторы
TARGET = 'price'

X, y = df.drop(columns=TARGET), df[TARGET]

Разделите выборку на обучающую и тестовую (объём тестовой возьмите равным 0.33), значение random_state должно быть равно 42.

Теперь реализуйте алгоритм линейной регрессии со стохастическим градиентным спуском (класс SGDRegressor). 

Отберите с помощью GridSearchCV оптимальные параметры по следующей сетке:

In [61]:
param_grid = {'loss': ['squared_error', 'epsilon_insensitive'],
              'penalty': ['elasticnet'],
              'alpha': np.logspace(-3, 3, 10),
              'l1_ratio': np.linspace(0, 1, 10),
              'learning_rate': ['constant'],
              'eta0': np.logspace(-4, -1, 4)}

In [62]:
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error

from sklearn import set_config
set_config(transform_output='pandas')

In [63]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [64]:
model = SGDRegressor(
        random_state=42, # генератор случайных чисел
        # max_iter=100000 # количество итераций на сходимость
    )

grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=5,
    n_jobs=-1,
    scoring='neg_mean_squared_error'
)

%time grid_search.fit(X_train, y_train) 
y_train_pred = grid_search.predict(X_train)
y_test_pred = grid_search.predict(X_test)

print(f'Наилучшие значения гиперпараметров: {grid_search.best_params_}')
print(f'Наилучшее значение метрики на валидационной выборке: {grid_search.best_score_}')

CPU times: total: 33.1 s
Wall time: 51.9 s
Наилучшие значения гиперпараметров: {'alpha': 0.001, 'eta0': 0.001, 'l1_ratio': 0.0, 'learning_rate': 'constant', 'loss': 'epsilon_insensitive', 'penalty': 'elasticnet'}
Наилучшее значение метрики на валидационной выборке: -0.04481974544218573


In [65]:
print(f'MSE on train: {mean_squared_error(y_train, y_train_pred):.3f}')
print(f'MSE on test: {mean_squared_error(y_test, y_test_pred):.3f}')

MSE on train: 0.045
MSE on test: 0.044


### Задание 3.1

Найдите третий корень полинома, взяв за точку старта 0.7. Введите получившееся значение с точностью до трёх знаков после точки-разделителя.

Попробуйте реализовать алгоритм с использованием Python на основе алгоритма градиентного спуска, изученного в предыдущем модуле.

In [52]:
x = symbols('x')
f = 6 * x**5 - 5 * x**4 - 4 * x**3 + 3 * x**2
f

6*x**5 - 5*x**4 - 4*x**3 + 3*x**2

Определяю первую производную

In [53]:
f_diff = f.diff(x)
f_diff

30*x**4 - 20*x**3 - 12*x**2 + 6*x

In [54]:
x0 = 0.7
n = 3

for i in range(n): # ищу третий корень
    # для первого шага нужно инициировать переменную x1
    if i == 0:
        x1 = x0
    
    # скобки обязательны, иначе приоритет деления плывет
    x1 = x1 - (f.subs(x, x1) / f_diff.subs(x, x1))

In [55]:
print(f'Значение корня полинома после {n} итераций расчетов: {x1:.3f}')

Значение корня полинома после 3 итераций расчетов: 0.629


### Расчет с помощью градиентного спуска

In [56]:
def f_(x_val):
    return f.subs(x, x_val)

In [57]:
def grad(x_val):
    return f.diff(x).subs(x, x_val)

In [59]:
f_(0.7)

-0.0940799999999999

In [60]:
x0 = 0.7 # Начальная точка для запуска алгоритма
gamma = 0.0000001 # Темп обучения изначально 0.1 и очень большой, сразу выпрыгиваю из впадины и уже туда не возвращаюсь
x_cur = x0 # Сохраняю текущую координату

vals = []
coords = []

i = 0
while True:
    # получение новой точки на основе предыдущей и градиента, с минусом т.к. минимизируем функцию и используем антиградиент
    x_new = x_cur - gamma * grad(x_cur)
    
    # Обучение прерывается только по количеству итераций
    if i > 200:
        break
    
    # сохраняю последнее найденное значение точки в переменную и добавляю в список
    x_cur = x_new
    coords.append(x_cur)
    # По точке считаю значение функции и сохраняю в список
    vals.append(f_(x_cur))
    
    i += 1 # обновляю счетчик
    
    # Вывожу данные на каждой итерации
    print(f"iter={i}; x={x_cur:.4f};"
          f" f(x)={f_(x_cur):.4f}; grad f(x)={grad(x_cur):.4f}")

iter=1; x=0.7000; f(x)=-0.0941; grad f(x)=-1.3370
iter=2; x=0.7000; f(x)=-0.0941; grad f(x)=-1.3370
iter=3; x=0.7000; f(x)=-0.0941; grad f(x)=-1.3370
iter=4; x=0.7000; f(x)=-0.0941; grad f(x)=-1.3370
iter=5; x=0.7000; f(x)=-0.0941; grad f(x)=-1.3370
iter=6; x=0.7000; f(x)=-0.0941; grad f(x)=-1.3370
iter=7; x=0.7000; f(x)=-0.0941; grad f(x)=-1.3370
iter=8; x=0.7000; f(x)=-0.0941; grad f(x)=-1.3370
iter=9; x=0.7000; f(x)=-0.0941; grad f(x)=-1.3370
iter=10; x=0.7000; f(x)=-0.0941; grad f(x)=-1.3370
iter=11; x=0.7000; f(x)=-0.0941; grad f(x)=-1.3370
iter=12; x=0.7000; f(x)=-0.0941; grad f(x)=-1.3370
iter=13; x=0.7000; f(x)=-0.0941; grad f(x)=-1.3370
iter=14; x=0.7000; f(x)=-0.0941; grad f(x)=-1.3370
iter=15; x=0.7000; f(x)=-0.0941; grad f(x)=-1.3370
iter=16; x=0.7000; f(x)=-0.0941; grad f(x)=-1.3370
iter=17; x=0.7000; f(x)=-0.0941; grad f(x)=-1.3370
iter=18; x=0.7000; f(x)=-0.0941; grad f(x)=-1.3370
iter=19; x=0.7000; f(x)=-0.0941; grad f(x)=-1.3370
iter=20; x=0.7000; f(x)=-0.0941; grad f(

### Метод Ньютона для оптимизации функции

In [61]:
f = x ** 3 - 3 * x ** 2 - 45 * x + 40
f

x**3 - 3*x**2 - 45*x + 40

Нахожу первую и вторую производные

In [64]:
f_diff = f.diff(x)
f_diff

3*x**2 - 6*x - 45

In [65]:
f_diff2 = f.diff(x, 2)
f_diff2

6*(x - 1)

In [66]:
def func1(x):
    return 3 * x ** 2 - 6 * x - 45

def func2(x):
    return 6 * x - 6

In [70]:
initial_value = 42
iter_count = 0
x_curr = initial_value
epsilon = 0.0001
f = func1(x_curr)

while (abs(f) > epsilon):
    f = func1(x_curr)
    f_prime = func2(x_curr)
    x_curr = x_curr - (f)/(f_prime)
    iter_count += 1
    print(f'{iter_count}: {x_curr}')

1: 21.695121951219512
2: 11.734125501243229
3: 7.1123493600499685
4: 5.365000391507974
5: 5.015260627016227
6: 5.000029000201801
7: 5.000000000105126
8: 5.000000000000001


Можно объединить всё в одну функцию

In [71]:
def newtons_method(f, fprime, x0, tol=0.0001):
    iter_count = 0
    x_curr = x0
    f_val = f(x_curr)
    
    while (abs(f_val) > tol):
        f_val = f(x_curr)
        f_prime_val = fprime(x_curr)
        x_curr = x_curr - (f_val)/(f_prime_val)
        iter_count += 1
    return x_curr

newtons_method(f=func1, fprime=func2, x0=42, tol=0.0001)

5.000000000000001

А можно воспользоваться реализацией из библиотеки scipy

In [72]:
from scipy.optimize import newton
newton(func=func1, fprime=func2, x0=50, tol=0.0001)

5.0

### Задание 3.6

Дана функция f. Найдите решение уравнения f(x) = 0 для поиска корня в окрестностях точки x0=12. Ответ округлите до трёх знаков после точки-разделителя.

In [32]:
x = symbols('x')
f = x**3 - 72 * x - 220
f

x**3 - 72*x - 220

Определяю первую производную

In [33]:
f_diff = f.diff(x)
f_diff

3*x**2 - 72

In [34]:
x0 = 12
max_iterations = 10
current_iteration = 0
accuracy_threshold = 0.00001
f_x1 = 0

x1 = x0

for current_iteration in range(max_iterations):
    print(f'{current_iteration+1} : x1 = {x1}, f(x1) = {f.subs(x, x1)}')
    
    x1 = x1 - (f.subs(x, x1) / f_diff.subs(x, x1))
    x1 = float(x1)

    # Условие ранней остановки, если значение функции изменяется к предыдущему менее чем на accuracy_threshold
    if abs(f_x1 - f.subs(x, x1)) < accuracy_threshold:
        break

    # Обновляю значение функции для текущего x1
    f_x1 = f.subs(x, x1)

1 : x1 = 12, f(x1) = 644
2 : x1 = 10.21111111111111, f(x1) = 109.479779149520
3 : x1 = 9.756461564762278, f(x1) = 6.23812134825107
4 : x1 = 9.727252176332618, f(x1) = 0.0249473775788829
5 : x1 = 9.72713442131889, f(x1) = 4.04639536100149E-7


In [35]:
print(f'Значение корня полинома после {current_iteration} итераций расчетов: {x1:.3f}')

Значение корня полинома после 4 итераций расчетов: 9.727


### Задание 3.7

Найдите положительный корень для уравнения f(x) = 0.

В качестве стартовой точки возьмите x0 = 2.2.

Расчёт произведите поэтапно или с помощью Python.

Ответ округлите до двух знаков после точки-разделителя.

In [36]:
x = symbols('x')
f = x**2 + 9 * x - 5
f

x**2 + 9*x - 5

Определяю первую производную

In [37]:
f_diff = f.diff(x)
f_diff

2*x + 9

In [38]:
x0 = 2.2
max_iterations = 10
current_iteration = 0
accuracy_threshold = 0.00001
f_x1 = 0

x1 = x0

for current_iteration in range(max_iterations):
    print(f'{current_iteration+1} : x1 = {x1}, f(x1) = {f.subs(x, x1)}')
    
    x1 = x1 - (f.subs(x, x1) / f_diff.subs(x, x1))
    x1 = float(x1)

    # Условие ранней остановки, если значение функции изменяется к предыдущему менее чем на accuracy_threshold
    if abs(f_x1 - f.subs(x, x1)) < accuracy_threshold:
        break

    # Обновляю значение функции для текущего x1
    f_x1 = f.subs(x, x1)

1 : x1 = 2.2, f(x1) = 19.6400000000000
2 : x1 = 0.7343283582089555, f(x1) = 2.14819336155046
3 : x1 = 0.5291259698087832, f(x1) = 0.0421080202051343
4 : x1 = 0.5249395544696249, f(x1) = 0.0000175260733925597
5 : x1 = 0.5249378105607477, f(x1) = 3.04112290905323E-12


In [39]:
print(f'Значение корня полинома после {current_iteration} итераций расчетов: {x1:.2f}')

Значение корня полинома после 4 итераций расчетов: 0.52


### Задание 3.9

С помощью метода Ньютона найдите точку минимума для функции f(x).

Для расчётов используйте Python.

В качестве стартовой точки возьмите x0 = 42, точность примите за 0.0001.

Ответ округлите до трёх знаков после точки-разделителя.

In [41]:
x = symbols('x')
f = 8 * x ** 3 - 2 * x ** 2 - 450
f

8*x**3 - 2*x**2 - 450

Нахожу первую и вторую производные

In [42]:
f_diff = f.diff(x)
f_diff

24*x**2 - 4*x

In [43]:
f_diff2 = f.diff(x, 2)
f_diff2

4*(12*x - 1)

In [44]:
def diff1(x):
    return 24 * x ** 2 - 4 * x

def diff2(x):
    return 4 * (12 * x - 1)

In [45]:
initial_value = 42
iter_count = 0
x_curr = initial_value
epsilon = 0.0001
f = diff1(x_curr)

while (abs(f) > epsilon):
    f = diff1(x_curr)
    
    x_curr = x_curr - (f)/(diff2(x_curr))
    iter_count += 1
    
    print(f'{iter_count}: {x_curr}')

1: 21.041749502982107
2: 10.562707090133793
3: 5.323351550447383
4: 2.7040050774153417
5: 1.3949941413301903
6: 0.7418109325525483
7: 0.41784523900811205
8: 0.26096925221473555
9: 0.19169814030401197
10: 0.16955770984744145
11: 0.1667151339969682
12: 0.1666666807529666
13: 0.16666666666666785


In [46]:
print(f'Значение точки минимума после {iter_count} итераций расчетов: {x_curr:.3f}')

Значение точки минимума после 13 итераций расчетов: 0.167
